In [2]:
import re
import pandas as pd
import requests 
from bs4 import BeautifulSoup 

In [6]:
municipios=pd.read_excel("links_municipios.xlsx")

In [7]:
municipios.head()

,URL MUNICIPIO,MUNICIPIO,DEPARTAMENTO
0,http://plebiscito.registraduria.gov.co/99PL/DP...,EL ENCANTO,AMAZONAS
1,http://plebiscito.registraduria.gov.co/99PL/DP...,LA CHORRERA,AMAZONAS
2,http://plebiscito.registraduria.gov.co/99PL/DP...,LA PEDRERA,AMAZONAS
3,http://plebiscito.registraduria.gov.co/99PL/DP...,LA VICTORIA,AMAZONAS
4,http://plebiscito.registraduria.gov.co/99PL/DP...,LETICIA,AMAZONAS


In [8]:
len(municipios)

1122

In [9]:
municipios['link_real']=municipios['URL MUNICIPIO'].map(lambda x:x.replace('http://plebiscito.registraduria.gov.co/', 'https://elecciones.registraduria.gov.co/pre_plebis_2016/'))

In [10]:
municipios.head()

,URL MUNICIPIO,MUNICIPIO,DEPARTAMENTO,link_real
0,http://plebiscito.registraduria.gov.co/99PL/DP...,EL ENCANTO,AMAZONAS,https://elecciones.registraduria.gov.co/pre_pl...
1,http://plebiscito.registraduria.gov.co/99PL/DP...,LA CHORRERA,AMAZONAS,https://elecciones.registraduria.gov.co/pre_pl...
2,http://plebiscito.registraduria.gov.co/99PL/DP...,LA PEDRERA,AMAZONAS,https://elecciones.registraduria.gov.co/pre_pl...
3,http://plebiscito.registraduria.gov.co/99PL/DP...,LA VICTORIA,AMAZONAS,https://elecciones.registraduria.gov.co/pre_pl...
4,http://plebiscito.registraduria.gov.co/99PL/DP...,LETICIA,AMAZONAS,https://elecciones.registraduria.gov.co/pre_pl...


In [6]:
def busqueda(municipio):
    html = requests.get(municipio, verify=False).text
    # estructurar datos a partir de archivos HTML
    soup = BeautifulSoup(html, "lxml")
    a=re.findall('<div class="cajaDatosHeader"><b>[\w\s+%.-]+|<div class="cajaDatosBody"><span class="porcentajesCajas">[\w,+%.-]+|</span><br/><span class="descripcionCaja">[\w,\s%.-]+', str(soup))
    ba=[]
    ba.append(a[1])
    bb=[]
    bb.append(a[2])
    m=re.search("([\w]+) de ([\w]+)", str(bb))
    bc=[]
    bc.append(a[4])
    bd=[]
    bd.append(a[5])
    v1=re.findall("[\d.]+", str(bd))[0]
    v2=re.findall("[\d.]+", str(bd))[1]
    b=re.findall('<div class="skill-bar-percent pVotos">[\w,%()\s]+|<div class="skill-bar-percent">[\w,()%\s]+|<div class="skillbar-title"><span>[\w(),%\s]+', str(soup))
    be=[]
    be.append(b[1])
    app=re.findall("[\d,%.]+", str(be))

    bf=[]
    bf.append(b[2])
    ap=re.search("[\d.]+", str(bf))

    bg=[]
    bg.append(b[4])
    noapp=re.findall("[\d,%.]+", str(bg))

    bh=[]
    bh.append(b[5])
    noap=re.search("[\d.]+", str(bh))
    c=re.findall('<div class="contenido"><b>[\w\s¡]+|</b><br/>[\d.]+', str(soup))
    ca=[]
    ca.append(c[1])
    nu1=re.findall("[\d.]+", str(ca))
    cb=[]
    cb.append(c[3])
    nu2=re.findall("[\d.]+", str(cb))
    cc=[]
    cc.append(c[5])
    nu3=re.findall("[\d.]+", str(cc))
    
    base=pd.DataFrame()
    
    base['link_real']=''
    base['nombre_muni']=''
    base['nombre_dep']=''
    base['mesas_informadas%']=ba
    base['mesas_informadas%']=base['mesas_informadas%'].map(lambda x:x.replace('<div class="cajaDatosBody"><span class="porcentajesCajas">',""))

    base['mesas_usadas']=m.group(1)
    base['mesas_disponibles']=m.group(2)

    base['votanes%']=bc
    base['votanes%']=base['votanes%'].map(lambda x:x.replace('<div class="cajaDatosBody"><span class="porcentajesCajas">',""))
    try:
        base['votanes_cumplieron']=v1
        base['votanes_habilidatos']=v2
    except:
        base['votanes_cumplieron']=None
        base['votanes_habilidatos']=None
    base['apoya%']=app
    base['apoya#']=ap.group(0)

    base['no_apoya%']=noapp
    base['no_apoya#']=noap.group(0)

    base['votos_validos']=nu1
    base['votos_no_marcador']=nu2
    base['votos_anullados%']=nu3
    
    
    return base


In [7]:
busqueda(municipios['link_real'][1])
#municipios['link_real'][1]

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,link_real,nombre_muni,nombre_dep,mesas_informadas%,mesas_usadas,mesas_disponibles,votanes%,votanes_cumplieron,votanes_habilidatos,apoya%,apoya#,no_apoya%,no_apoya#,votos_validos,votos_no_marcador,votos_anullados%
0,NaN,NaN,NaN,100%,3,3,"35,53%",366,1.030,"82,87%",300,"17,12%",62,362,1,3


In [8]:
base_total=pd.DataFrame()
for i in range(len(municipios['link_real'])):
    base=busqueda(municipios['link_real'][i])
    base_total=base_total.append(base).reset_index(drop=True)
    base_total['link_real'][i]=municipios['link_real'][i]
    base_total['nombre_muni'][i]=municipios['MUNICIPIO'][i]
    base_total['nombre_dep'][i]=municipios['DEPARTAMENTO'][i]

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

...

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\lenovo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elecciones.registraduria.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  Inse

In [9]:
base_total

,link_real,nombre_muni,nombre_dep,mesas_informadas%,mesas_usadas,mesas_disponibles,votanes%,votanes_cumplieron,votanes_habilidatos,apoya%,apoya#,no_apoya%,no_apoya#,votos_validos,votos_no_marcador,votos_anullados%
0,https://elecciones.registraduria.gov.co/pre_pl...,EL ENCANTO,AMAZONAS,100%,2,2,"30,01%",223,743,"76,92%",170,"23,07%",51,221,0,2
1,https://elecciones.registraduria.gov.co/pre_pl...,LA CHORRERA,AMAZONAS,100%,3,3,"35,53%",366,1.030,"82,87%",300,"17,12%",62,362,1,3
2,https://elecciones.registraduria.gov.co/pre_pl...,LA PEDRERA,AMAZONAS,100%,3,3,"21,42%",216,1.008,"88,31%",189,"11,68%",25,214,1,1
3,https://elecciones.registraduria.gov.co/pre_pl...,LA VICTORIA,AMAZONAS,100%,1,1,"27,77%",5,18,"80,00%",4,"20,00%",1,5,0,0
4,https://elecciones.registraduria.gov.co/pre_pl...,LETICIA,AMAZONAS,100%,82,82,"25,98%",8.990,34.592,"49,66%",4,"50,33%",4,8.866,40,84
5,https://elecciones.registraduria.gov.co/pre_pl...,MIRITI PARANA,AMAZONAS,100%,2,2,"20,12%",96,477,"92,70%",89,"7,29%",7,96,0,0
6,https://elecciones.registraduria.gov.co/pre_pl...,PUERTO ALEGRIA,AMAZONAS,100%,1,1,"22,77%",59,259,"91,22%",52,"8,77%",5,57,1,1
7,https://elecciones.registraduria.gov.co/pre_pl...,PUERTO ARICA,AMAZONAS,100%,1,1,"29,38%",119,405,"81,51%",97,"18,48%",22,119,0,0
8,https://elecciones.registraduria.gov.co/pre_pl...,PUERTO NARIÑO,AMAZONAS,100%,10,10,"26,70%",1.153,4.317,"75,18%",830,"24,81%",274,1.104,32,17
9,https://elecciones.registraduria.gov.co/pre_pl...,PUERTO SANTANDER,AMAZONAS,100%,2,2,"22,27%",143,642,"88,65%",125,"11,34%",16,141,0,2


In [10]:
base_total.to_excel("base_total.xlsx")